In [ ]:
import pandas as pd
import swifter
from datetime import datetime
import datetime
import numpy as np

### language, tweet_timestamp, enaged_with_user_id, engaged_with_user_follower_count, engaged_with_user_following_count, enaging_user_id, 

### enaging_user_follower_count, enaging_user_following_count, retweet_with_comment_timestamp

In [ ]:
def ep_to_day(ep):
    return datetime.datetime.fromtimestamp(ep).weekday()

In [ ]:
names = ['language', 'tweet_timestamp', 'enaged_with_user_id', 'engaged_with_user_follower_count',\
         'engaged_with_user_following_count', 'enaging_user_id', 'enaging_user_follower_count',\
         'enaging_user_following_count', 'retweet_with_comment_timestamp']

training = pd.read_csv('s3://recsys-challenge-2020/training.tsv', usecols = [7, 8, 9, 10, 11, 14, 15, 16, 22],
                       names = names, encoding="utf-8", sep="\x01")

In [ ]:
training.to_csv('/home/ubuntu/recsys_challenge_2020/classification_type_models/f3/training.csv', index = False)

In [ ]:
training['day_of_tweet'] = training.swifter.allow_dask_on_strings(enable=True).apply(lambda x: ep_to_day(x['tweet_timestamp'])\
                                        , axis  = 1)

In [ ]:
training.to_csv('/home/ubuntu/recsys_challenge_2020/classification_type_models/f3/training_day.csv', index = False)

In [ ]:
from sklearn import preprocessing

x = training[['engaged_with_user_follower_count', "engaged_with_user_following_count",\
                "enaging_user_follower_count", "enaging_user_following_count"]].values #returns a numpy array

In [ ]:
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df = pd.DataFrame(x_scaled)
training_scaled = pd.concat([training, df], axis = 1)

In [ ]:
training_scaled

In [ ]:
training_scaled['retweet_with_comment_bool'] = training_scaled.retweet_with_comment_timestamp.fillna(0)
training_scaled.loc[training_scaled.retweet_with_comment_bool != 0.0, 'retweet_with_comment_bool'] = 1.0

In [ ]:
training.retweet_with_comment_bool.value_counts()

In [ ]:
training_scaled.rename(columns = {0:'engaged_with_user_follower_count_scaled'\
                     ,1:'engaged_with_user_following_count_scaled'\
                     ,2:'enaging_user_follower_count_scaled',\
                     3:'enaging_user_following_count_scaled'},inplace = True)

In [ ]:
training_scaled.columns

In [ ]:
training_scaled[['language', 'enaged_with_user_id', 'enaging_user_id', 'day_of_tweet',\
                'engaged_with_user_follower_count_scaled', 'engaged_with_user_following_count_scaled',\
               'enaging_user_follower_count_scaled', 'enaging_user_following_count_scaled',\
                'retweet_with_comment_bool']].to_csv('/home/ubuntu/recsys_challenge_2020/classification_type_models/f3/training_scaled.csv', index = False)

In [ ]:
training_data = training_scaled[['language', 'enaged_with_user_id', 'enaging_user_id', 'day_of_tweet',\
                'engaged_with_user_follower_count_scaled', 'engaged_with_user_following_count_scaled',\
               'enaging_user_follower_count_scaled', 'enaging_user_following_count_scaled',\
                'retweet_with_comment_bool']]

In [ ]:
training_data.head()

In [ ]:
training_data.set_index(keys=['day_of_tweet'], drop=False,inplace=True)
names = training_data['day_of_tweet'].unique().tolist()
train_scratch = pd.DataFrame()
val_scratch = pd.DataFrame()
for day in names:
    print(day)
    train_day = training_data.loc[training_data.day_of_tweet==day]
    n_head = 90
    n_tail = 10
    train_set_scratch = train_day.head(int(len(train_day)*(n_head/100)))
    val_set_scratch = train_day.tail(int(len(train_day)*(n_tail/100)))
    train_scratch = pd.concat([train_scratch, train_set_scratch])
    val_scratch = pd.concat([val_scratch, val_set_scratch])

In [ ]:
train_scratch.dtypes

In [ ]:
val_scratch.dtypes

In [ ]:
val_scratch.reset_index(drop=True, inplace=True)

In [ ]:
train_scratch.reset_index(drop=True, inplace=True)

In [ ]:
train_scratch.to_csv('/home/ubuntu/recsys_challenge_2020/classification_type_models/f3/train_scratch.csv', index = False)
val_scratch.to_csv('/home/ubuntu/recsys_challenge_2020/classification_type_models/f3/val_scratch.csv', index = False)

In [ ]:
train_scratch = pd.read_csv('/home/ubuntu/recsys_challenge_2020/classification_type_models/f3/train_scratch.csv')

In [ ]:
val_scratch = pd.read_csv('/home/ubuntu/recsys_challenge_2020/classification_type_models/f3/val_scratch.csv')

In [ ]:
data = pd.concat([train_scratch,val_scratch],keys=[0,1])

In [ ]:
data.head()

In [ ]:
data.dtypes

In [ ]:
data['day_of_tweet'] = data['day_of_tweet'].astype(str)

In [ ]:
class LibffmConverter:
    """Converts an input dataframe to another dataframe in libffm format. A text file of the converted
    Dataframe is optionally generated.
    .. note::
        The input dataframe is expected to represent the feature data in the following schema:
        .. code-block:: python
            |field-1|field-2|...|field-n|rating|
            |feature-1-1|feature-2-1|...|feature-n-1|1|
            |feature-1-2|feature-2-2|...|feature-n-2|0|
            ...
            |feature-1-i|feature-2-j|...|feature-n-k|0|
        Where
        1. each `field-*` is the column name of the dataframe (column of label/rating is excluded), and
        2. `feature-*-*` can be either a string or a numerical value, representing the categorical variable or
        actual numerical variable of the feature value in the field, respectively.
        3. If there are ordinal variables represented in int types, users should make sure these columns
        are properly converted to string type.
        The above data will be converted to the libffm format by following the convention as explained in
        `this paper <https://www.csie.ntu.edu.tw/~r01922136/slides/ffm.pdf>`_.
        i.e. `<field_index>:<field_feature_index>:1` or `<field_index>:<field_index>:<field_feature_value>`, depending on
        the data type of the features in the original dataframe.
    Args:
        filepath (str): path to save the converted data.
    Attributes:
        field_count (int): count of field in the libffm format data
        feature_count (int): count of feature in the libffm format data
        filepath (str or None): file path where the output is stored - it can be None or a string
    Examples:
        >>> import pandas as pd
        >>> df_feature = pd.DataFrame({
                'rating': [1, 0, 0, 1, 1],
                'field1': ['xxx1', 'xxx2', 'xxx4', 'xxx4', 'xxx4'],
                'field2': [3, 4, 5, 6, 7],
                'field3': [1.0, 2.0, 3.0, 4.0, 5.0],
                'field4': ['1', '2', '3', '4', '5']
            })
        >>> converter = LibffmConveter().fit(df_feature, col_rating='rating')
        >>> df_out = converter.transform(df_feature)
        >>> df_out
            rating field1 field2   field3 field4
        0       1  1:1:1  2:4:3  3:5:1.0  4:4:1
        1       0  1:2:1  2:4:4  3:5:2.0  4:5:1
        2       0  1:3:1  2:4:5  3:5:3.0  4:6:1
        3       1  1:3:1  2:4:6  3:5:4.0  4:7:1
        4       1  1:3:1  2:4:7  3:5:5.0  4:8:1
    """

    def __init__(self, filepath=None):
        self.filepath = filepath
        self.col_rating = None
        self.field_names = None
        self.field_count = None
        self.feature_count = None

    def fit(self, df, col_rating=3):
        """Fit the dataframe for libffm format.
        This method does nothing but check the validity of the input columns
        Args:
            df (pd.DataFrame): input Pandas dataframe.
            col_rating (str): rating of the data.
        Return:
            obj: the instance of the converter
        """

        # Check column types.
        types = df.dtypes
        if not all(
            [
                x == object or np.issubdtype(x, np.integer) or x == np.float
                for x in types
            ]
        ):
            raise TypeError("Input columns should be only object and/or numeric types.")

        if col_rating not in df.columns:
            raise TypeError(
                "Column of {} is not in input dataframe columns".format(col_rating)
            )

        self.col_rating = col_rating
        self.field_names = list(df.drop(col_rating, axis=1).columns)

        return self

    def transform(self, df):
        """Tranform an input dataset with the same schema (column names and dtypes) to libffm format 
        by using the fitted converter.
        Args: 
            df (pd.DataFrame): input Pandas dataframe.
        Return:
            pd.DataFrame: output libffm format dataframe.
        """
        if self.col_rating not in df.columns:
            raise ValueError(
                "Input dataset does not contain the label column {} in the fitting dataset".format(
                    self.col_rating
                )
            )

        if not all([x in df.columns for x in self.field_names]):
            raise ValueError(
                "Not all columns in the input dataset appear in the fitting dataset"
            )

        # Encode field-feature.
        idx = 1
        self.field_feature_dict = {}
        for field in self.field_names:
            for feature in df[field].values:
                # Check whether (field, feature) tuple exists in the dict or not.
                # If not, put them into the key-values of the dict and count the index.
                if (field, feature) not in self.field_feature_dict:
                    self.field_feature_dict[(field, feature)] = idx
                    if df[field].dtype == object:
                        idx += 1
            if df[field].dtype != object:
                idx += 1

        self.field_count = len(self.field_names)
        self.feature_count = idx - 1

        def _convert(field, feature, field_index, field_feature_index_dict):
            field_feature_index = field_feature_index_dict[(field, feature)]
            if isinstance(feature, str):
                feature = 1
            return "{}:{}:{}".format(field_index, field_feature_index, feature)

        for col_index, col in enumerate(self.field_names):
            df[col] = df[col].apply(
                lambda x: _convert(col, x, col_index + 1, self.field_feature_dict)
            )

        # Move rating column to the first.
        column_names = self.field_names[:]
        column_names.insert(0, self.col_rating)
        df = df[column_names]

        if self.filepath is not None:
            np.savetxt(self.filepath, df.values, delimiter=" ", fmt="%s")

        return df

    def fit_transform(self, df, col_rating=4):
        """Do fit and transform in a row
        Args:
            df (pd.DataFrame): input Pandas dataframe.
            col_rating (str): rating of the data.
        Return:
            pd.DataFrame: output libffm format dataframe.
        """
        return self.fit(df, col_rating=col_rating).transform(df)

    def get_params(self):
        """Get parameters (attributes) of the libffm converter
        Return:
            dict: parameters field count, feature count, and file path.
        """
        return {
            "field count": self.field_count,
            "feature count": self.feature_count,
            "file path": self.filepath,
        }


    def negative_feedback_sampler(
        df,
        col_user=0,
        col_item=1,
        col_label=1,
        ratio_neg_per_user=1,
        seed=42,
    ):
        """Utility function to sample negative feedback from user-item interaction dataset.
        This negative sampling function will take the user-item interaction data to create 
        binarized feedback, i.e., 1 and 0 indicate positive and negative feedback, 
        respectively. 
        Negative sampling is used in the literature frequently to generate negative samples 
        from a user-item interaction data.
        See for example the `neural collaborative filtering paper <https://www.comp.nus.edu.sg/~xiangnan/papers/ncf.pdf>`_.

        Args:
            df (pandas.DataFrame): input data that contains user-item tuples.
            col_user (str): user id column name.
            col_item (str): item id column name.
            col_label (str): label column name. It is used for the generated columns where labels
            of positive and negative feedback, i.e., 1 and 0, respectively, in the output dataframe.
            ratio_neg_per_user (int): ratio of negative feedback w.r.t to the number of positive feedback for each user. 
            If the samples exceed the number of total possible negative feedback samples, it will be reduced to the number
            of all the possible samples.
            seed (int): seed for the random state of the sampling function.
        Returns:
            pandas.DataFrame: data with negative feedback 
        Examples:
            >>> import pandas as pd
            >>> df = pd.DataFrame({
                'userID': [1, 2, 3],
                'itemID': [1, 2, 3],
                'rating': [5, 5, 5]
            })
            >>> df_neg_sampled = negative_feedback_sampler(
                df, col_user='userID', col_item='itemID', ratio_neg_per_user=1
            )
            >>> df_neg_sampled
            userID  itemID  feedback
            1   1   1
            1   2   0
            2   2   1
            2   1   0
            3   3   1
            3   1   0
        """
        # Get all of the users and items.
        users = df[col_user].unique()
        items = df[col_item].unique()

        # Create a dataframe for all user-item pairs
        df_neg = user_item_pairs(
            pd.DataFrame(users, columns=[col_user]),
            pd.DataFrame(items, columns=[col_item]),
            user_item_filter_df=df,
        )
        df_neg[col_label] = 0

        df_pos = df.copy()
        df_pos[col_label] = 1

        df_all = pd.concat([df_pos, df_neg], ignore_index=True, sort=True)
        df_all = df_all[[col_user, col_item, col_label]]

        # Sample negative feedback from the combined dataframe.
        df_sample = (
            df_all.groupby(col_user)
            .apply(
                lambda x: pd.concat(
                    [
                        x[x[col_label] == 1],
                        x[x[col_label] == 0].sample(
                            min(
                                max(
                                    round(len(x[x[col_label] == 1]) * ratio_neg_per_user), 1
                                ),
                                len(x[x[col_label] == 0]),
                            ),
                            random_state=seed,
                            replace=False,
                        )
                        if len(x[x[col_label] == 0] > 0)
                        else pd.DataFrame({}, columns=[col_user, col_item, col_label]),
                    ],
                    ignore_index=True,
                    sort=True,
                )
            )
            .reset_index(drop=True)
            .sort_values(col_user)
        )

        return df_sample


    def has_columns(df, columns):
        """Check if DataFrame has necessary columns
        Args:
            df (pd.DataFrame): DataFrame
            columns (list(str): columns to check for
        Returns:
            bool: True if DataFrame has specified columns
        """

        result = True
        for column in columns:
            if column not in df.columns:
                logger.error("Missing column: {} in DataFrame".format(column))
                result = False

        return result


    def has_same_base_dtype(df_1, df_2, columns=None):
        """Check if specified columns have the same base dtypes across both DataFrames
        Args:
            df_1 (pd.DataFrame): first DataFrame
            df_2 (pd.DataFrame): second DataFrame
            columns (list(str)): columns to check, None checks all columns
        Returns:
            bool: True if DataFrames columns have the same base dtypes
        """

        if columns is None:
            if any(set(df_1.columns).symmetric_difference(set(df_2.columns))):
                logger.error(
                    "Cannot test all columns because they are not all shared across DataFrames"
                )
                return False
            columns = df_1.columns

        if not (
            has_columns(df=df_1, columns=columns) and has_columns(df=df_2, columns=columns)
        ):
            return False

        result = True
        for column in columns:
            if df_1[column].dtype.type.__base__ != df_2[column].dtype.type.__base__:
                logger.error("Columns {} do not have the same base datatype".format(column))
                result = False

        return result

In [ ]:
converter = LibffmConverter().fit(data, col_rating='retweet_with_comment_bool')

In [ ]:
del [[train_scratch, val_scratch]]

In [ ]:
import gc
gc.collect()

In [ ]:
train_scratch = pd.DataFrame()

In [ ]:
val_scratch = pd.DataFrame()

In [ ]:
data

In [ ]:
df_out = converter.transform(data)

In [ ]:
train_ffm, test_ffm = df_out.xs(0),df_out.xs(1)

In [ ]:
train_ffm

In [ ]:
train_ffm.to_csv('/home/ubuntu/recsys_challenge_2020/classification_type_models/f3/train_set.ffm',\
                 index = False, header = False, sep = ' ')

In [ ]:
test_ffm

In [ ]:
test_ffm.to_csv('/home/ubuntu/recsys_challenge_2020/classification_type_models/f3/val_set.ffm',\
                index = False, header = False, sep = ' ')

# Feature Indexes
"text_ tokens" = 0
"hashtags" = 1
"tweet_id" = 2
"present_media" = 3
"present_links" = 4
"present_domains" = 5
"tweet_type" = 6
"language" = 7
"tweet_timestamp" = 8
"enaged_with_user_id" = 9
"engaged_with_user_follower_count" = 10
"engaged_with_user_following_count" = 11
"engaged_with_user_is_verified" = 12
"engaged_with_user_account_creation" = 13
"enaging_user_id" = 14
"enaging_user_follower_count" = 15
"enaging_user_following_count" = 16
"enaging_user_is_verified" = 17
"enaging_user_account_creation" = 18
"engagee_follows_engager" = 19
"reply_timestamp" = 20
"retweet_timestamp" = 21
"retweet_with_comment_timestamp" = 22
"like_timestamp" = 23